## Tal Sharoni, Justin Lipper

### Phys-321 Final Project

In [5]:
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.visualization import astropy_mpl_style
from scipy.integrate import trapz
import scipy.stats as stats
import corner
import emcee
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
